In [13]:
import pyodbc
import numpy as np

In [2]:
batch_size = 10

In [20]:
def sql_server_data_generator():
    cnxn = pyodbc.connect('Trusted_Connection=yes;DRIVER={SQL Server};SERVER=DESKTOP-1RHDOBR\GORDASQL;DATABASE=positive;UID=pyuser;PWD=pypypy')
    cursor = cnxn.cursor()
    cursor.execute("SELECT [id],[ttext] FROM [dbo].[sortpos]")
    
    X, y = [], []
    for row in cursor:
        r_id = row.id
        r_text = row.ttext
        X.append(r_id)
        y.append(r_text)
        
        if len(X) == batch_size:
            npx = np.array(X)
            npy = np.array(y)
            yield npx, npy
            X, y = [], []
    pyodbc.Connection.close(cnxn)

In [21]:
ssdg = sql_server_data_generator()

In [26]:
next(ssdg)

(array([410280177827344385, 410280180490727424, 410280218645917696,
        410280218746978304, 410280221682569216, 410280275781107712,
        410280277508751360, 410280390243254272, 410280390386253824,
        410280392017850368], dtype=int64),
 array([ '@rinka_pruf ноуноу,я ничего против Бенбекона не имею,наоборот... Просто хотела бы послушать Исина :D',
        '@alevtina667 Пока ты через \\"козла\\" прыгала,я тебе там Сашку фотографировала на память )',
        'С Добрым утром! Одевайтесь потеплее!) http://t.co/wdfJ2p5rff',
        'RT @GlukoZa_Ionova: Вообще никого не обижайте! Все пригодятся ))) доброе утро! ',
        'никакого новогоднего настроения...что то надо делать)) буду снежинки вырезать, офис разукрашивать',
        'RT @zodocoduzuxo: хороша, чертовка...а глаза, глаза какие!!! ;)',
        '@strilezkijslawa Доброе утро,Слава!У нас сегодня -13 уже)Завтра обещают до -18))',
        'Какая же она милая!!))Даже 10-лет спустя!:))) http://t.co/gERQ8wsSA1',
        'RT @glrrm